In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torch.optim as optim
import time
import copy

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

cuda:0


In [2]:
trans = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

data = torchvision.datasets.ImageFolder(root='../Images', transform=trans)
traindata, testvaldata = torch.utils.data.random_split(data, [12000, 8580])
valdata, testdata = torch.utils.data.random_split(testvaldata, [4290, 4290])
dset = {'train': traindata, 'val': valdata}

dataloaders = {x: torch.utils.data.DataLoader(dset[x], batch_size=16, shuffle=True) for x in ['train', 'val']}
dataset_sizes = {'train':12000, 'val':4290}
testloader = torch.utils.data.DataLoader(testdata, batch_size = 16, shuffle=True)

print('Finished data augmentation')

Finished data augmentation


In [3]:
net = models.resnet50(pretrained=True)
num = net.fc.in_features
net.fc = nn.Linear(num, 120)
net.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
num_epochs = 30

since = time.time()
best_model_wts = copy.deepcopy(net.state_dict())
best_acc = 0.0

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch + 1, num_epochs))
    print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        if phase == 'train':
            net.train()  # Set model to training mode
        else:
            net.eval()   # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                outputs = net(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        if phase == 'train':
            scheduler.step()

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = (running_corrects.double() / dataset_sizes[phase])*100

        print('{} Loss: {:.4f} Acc: {:.2f}%'.format(
            phase, epoch_loss, epoch_acc))

        # deep copy the model
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(net.state_dict())

    print()

time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))
print('Best val Acc: {:4f}%'.format(best_acc))

# load best model weights
net.load_state_dict(best_model_wts)

Epoch 1/30
----------
train Loss: 2.4468 Acc: 46.62%
val Loss: 0.9415 Acc: 80.23%

Epoch 2/30
----------
train Loss: 0.9116 Acc: 76.05%
val Loss: 0.6501 Acc: 87.67%

Epoch 3/30
----------
train Loss: 0.6159 Acc: 83.17%
val Loss: 0.6264 Acc: 87.98%

Epoch 4/30
----------
train Loss: 0.4389 Acc: 88.12%
val Loss: 0.5571 Acc: 89.70%

Epoch 5/30
----------
train Loss: 0.3215 Acc: 91.89%
val Loss: 0.5539 Acc: 90.22%

Epoch 6/30
----------
train Loss: 0.2382 Acc: 94.21%
val Loss: 0.5659 Acc: 89.85%

Epoch 7/30
----------
train Loss: 0.1782 Acc: 95.79%
val Loss: 0.5874 Acc: 89.30%

Epoch 8/30
----------
train Loss: 0.1111 Acc: 98.22%
val Loss: 0.5213 Acc: 91.22%

Epoch 9/30
----------
train Loss: 0.0963 Acc: 98.58%
val Loss: 0.5079 Acc: 91.88%

Epoch 10/30
----------
train Loss: 0.0861 Acc: 98.80%
val Loss: 0.5093 Acc: 91.60%

Epoch 11/30
----------
train Loss: 0.0778 Acc: 99.16%
val Loss: 0.5212 Acc: 91.08%

Epoch 12/30
----------
train Loss: 0.0731 Acc: 99.20%
val Loss: 0.5059 Acc: 92.03%

E

<All keys matched successfully>

In [5]:
PATH = './resnet50.pth'
torch.save(net.state_dict(), PATH)

In [10]:
net = models.resnet50(pretrained=True)
num = net.fc.in_features
net.fc = nn.Linear(num, 120)
PATH = './resnet50.pth'
net.load_state_dict(torch.load(PATH))
net.to(device)

In [10]:
import numpy as np
import matplotlib.pyplot as plt
f = open("../../classes.txt", "r")
preds = np.zeros((0, 120))
labs = np.zeros(0)
classmap = {}
for i in range(120):
    classs = f.readline().split(" ")
    classmap[i] = classs[0]

class_correct = list(0. for i in range(120))
class_total = list(0. for i in range(120))
correct = 0.0
total = 0.0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        p = outputs.cpu().numpy()
        preds = np.append(preds, p, axis=0)
        labs = np.append(labs, labels.cpu(), axis=0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        c = (predicted == labels).squeeze()
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1
meanacc1 = (100 * correct / total)
newmap = {}
for i in range(120):
    newmap[classmap[i]] = 100 * class_correct[i] / class_total[i]
    
i = 1
with open('cnn_accuracies.txt','w') as l:
    for k, v in newmap.items():
        l.write(k + '   {}\n'.format(v))
        i+=1
        
sortedmap = sorted(newmap.items(), key=lambda item: item[1], reverse=True)
for k, v in sortedmap:
    print('Accuracy of %30s : %10d %%' % (k, v))
print('Mean accuracy: %5f %%' % meanacc1)

Accuracy of                       papillon :        100 %
Accuracy of                   Afghan_hound :        100 %
Accuracy of                         Saluki :        100 %
Accuracy of             Bedlington_terrier :        100 %
Accuracy of                 Border_terrier :        100 %
Accuracy of                       Airedale :        100 %
Accuracy of         curly_coated_retriever :        100 %
Accuracy of                 Sussex_spaniel :        100 %
Accuracy of            Irish_water_spaniel :        100 %
Accuracy of                       komondor :        100 %
Accuracy of                  Saint_Bernard :        100 %
Accuracy of                     Pomeranian :        100 %
Accuracy of                           chow :        100 %
Accuracy of                       keeshond :        100 %
Accuracy of              Brabancon_griffon :        100 %
Accuracy of            African_hunting_dog :        100 %
Accuracy of                    Maltese_dog :         98 %
Accuracy of   

In [ ]:
top5classes = np.zeros((0, 5))
top5probs = np.zeros((0, 5))

for c in range(len(preds)):
    ind = np.argpartition(preds[c, :], -5)[-5:]
    top5classes = np.append(top5classes, np.reshape(ind, (1,5)), axis=0)
    top5probs = np.append(top5probs, np.reshape(preds[c, ind], (1,5)), axis=0)
    
print(top5classes)
print(top5probs)
print(top5classes.shape)
print(top5probs.shape)

In [ ]:
for j in range(len(top5classes)):
    top5probs[j,:], top5classes[j,:] = (list(t) for t in zip(*sorted(zip(top5probs[j,:], top5classes[j,:]), reverse=True)))
print(top5classes)
print(top5probs)
print(top5classes.shape)
print(top5probs.shape)

In [ ]:
incorrect = np.zeros(0, dtype = int)
for i in range(4290):
    if labs[i] != top5classes[i,0]:
        incorrect = np.append(incorrect, i)

incorrectlabs = labs[incorrect]
incorrect5classes = top5classes[incorrect, :]
incorrect5probs = top5probs[incorrect, :]

print(incorrect5classes, incorrectlabs)
print(incorrect5probs.shape)


In [ ]:
probs = np.zeros((0,5))
classes = np.zeros((0,5))
labels = np.zeros(0)

for i in range(len(incorrectlabs)):
    if incorrect5probs[i,0] > 12:
        probs = np.append(probs, np.reshape(incorrect5probs[i,:], (1, 5)), axis=0)
        classes = np.append(classes, np.reshape(incorrect5classes[i,:], (1,5)), axis=0)
        labels = np.append(labels, incorrectlabs[i])

for i in range(len(probs)):
    print('{}.  True label: {:3.0f}    Top 5 predictions: {:4.0f},{:4.0f},{:4.0f},{:4.0f},{:4.0f}    Top 5 confidences: {:6.2f},{:6.2f},{:6.2f},{:6.2f},{:6.2f}'.format(i+1, labels[i], classes[i,0], classes[i,1], classes[i,2], classes[i,3], classes[i,4], probs[i,0], probs[i,1], probs[i,2], probs[i,3], probs[i,4]))

In [ ]:
'''
BELOW are other CNNs, PERFORMS POORLY compared to resnet transfer
'''

In [ ]:
'''
THIS IS PYTORCH TUTORIAL NET (NOT GOOD FOR FINE GRAIN)
'''

# import torch.nn as nn
# import torch.nn.functional as F


# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(3, 60, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(60, 16, 5)
#         self.fc1 = nn.Linear(16 * 53*53, 1000)
#         self.fc2 = nn.Linear(1000, 120)
# #         self.fc3 = nn.Linear(84, 10)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = x.view(x.size(0), 16*53*53)
#         x = F.relu(self.fc1(x))
#         #x = F.relu(self.fc2(x))
#         x = self.fc2(x)
#         return x


# net = Net()
# net.to(device)

In [ ]:
'''
THIS IS BETTER NETWORK
'''

# class Net(nn.Module):
    
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(3, 16, 3)
#         self.conv2 = nn.Conv2d(16, 32, 3)
#         self.conv3 = nn.Conv2d(32, 64, 3)
#         self.conv4 = nn.Conv2d(64, 128, 3)
#         self.conv5 = nn.Conv2d(128, 256, 3)
#         self.fc1 = nn.Linear(256 * 6 * 6, 120)
        
#         self.max_pool = nn.MaxPool2d(2, 2,ceil_mode=True)
#         self.dropout = nn.Dropout(0.2)
#         self.conv_bn1 = nn.BatchNorm2d(224,3)
#         self.conv_bn2 = nn.BatchNorm2d(16)
#         self.conv_bn3 = nn.BatchNorm2d(32)
#         self.conv_bn4 = nn.BatchNorm2d(64)
#         self.conv_bn5 = nn.BatchNorm2d(128)
#         self.conv_bn6 = nn.BatchNorm2d(256)
    
#     def forward(self, x):
        
#         x = F.relu(self.conv1(x))
#         x = self.max_pool(x)
#         x = self.conv_bn2(x)
        
#         x = F.relu(self.conv2(x))
#         x = self.max_pool(x)
#         x = self.conv_bn3(x)
        
#         x = F.relu(self.conv3(x))
#         x = self.max_pool(x)
#         x = self.conv_bn4(x)
        
#         x = F.relu(self.conv4(x))
#         x = self.max_pool(x)
#         x = self.conv_bn5(x)
        
#         x = F.relu(self.conv5(x))
#         x = self.max_pool(x)
#         x = self.conv_bn6(x)
        
#         x = x.view(-1, 256 * 6 * 6)
        
#         x = self.dropout(x)
#         x = self.fc1(x)
#         return x

# net = Net()
# net.to(device)